In [43]:
import requests
from bs4 import BeautifulSoup
from sklearn.datasets import load_iris
import os
import datetime as dt
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.metrics import accuracy_score
import pandas_datareader as pdr
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale, minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
np.random.seed(144)

df = pd.read_excel(os.getcwd()+'\\'+"fine1.xlsx", skiprows = 7) # 모델 학습용 데이터
df_test = pd.read_excel(os.getcwd()+'\\'+"test.xlsx", skiprows = 12) # 최종 테스트 데이터 전처리 이전
df_name = pd.read_excel(os.getcwd()+'\\'+"name.xlsx") # 최종 모델 랭킹 나열시 이름

df_change=pd.DataFrame()
df_merger=pd.DataFrame()

for i in range(1400):
    if df_test.columns[i].find('시가')!=-1: #dataguide로 재무재표를 끌어오면 가로로 일렬 세움으로 세로로 일열 세우기 위해 짜놓은 코드
        df_trans = df_change.transpose()
        df_merger=pd.concat([df_merger,df_trans])
        df_change=pd.DataFrame()
    else:
        df_change=df_change.append(df_test[df_test.columns[i]], ignore_index=True)

df_merger.columns=['이자보상배율(배)','매출총이익증가율(전분기)(%)','영업이익증가율(전분기)(%)','ROIC(%)','ROA(당기순이익)(%)','EV/EBITDA(직전4분기)(배)']
pd.DataFrame(df_merger).to_excel("df_merger.xlsx")
print(df_merger)



   이자보상배율(배) 매출총이익증가율(전분기)(%) 영업이익증가율(전분기)(%) ROIC(%) ROA(당기순이익)(%)  \
0     259.98             3.71           -12.6   24.39         14.15   
0        NaN           -49.59              적지  -21.05         -4.42   
0     100.81             0.51           -1.25   26.14         16.49   
0      27.23           -10.15          -23.08   14.82           6.8   
0      29.43             8.58            2.69  196.67          13.2   
..       ...              ...             ...     ...           ...   
0      11.88            21.25           33.79   16.73          6.06   
0     590.32            28.64          673.01   31.74          4.05   
0       14.9            -4.26            19.0   47.34          9.73   
0     744.03             4.44            2.65     NaN         17.72   
0        1.6            -7.46          -44.76    1.92          0.46   

   EV/EBITDA(직전4분기)(배)  
0                 6.23  
0                  2.9  
0                 4.59  
0                85.46  
0                35.58

In [44]:
def z_score_normalize(lst):
    normalized = []
    for value in lst:
        normalized_num = (value - np.mean(lst)) / np.std(lst)
        normalized.append(normalized_num)
    return normalized

In [45]:
#학습 데이터 전처리
df222=z_score_normalize(df['이자보상배율(배)'])
df333=z_score_normalize(df['매출총이익증가율(전분기)(%)'])
df444=z_score_normalize(df['영업이익증가율(전분기)(%)'])
df555=z_score_normalize(df['ROIC(%)'])
df666=z_score_normalize(df['ROA(당기순이익)(%)'])
df777=z_score_normalize(df['EV/EBITDA(직전4분기)(배)'])
df999=df['시가(원)']


dfch2=np.reshape(df222, (-1, 1))
dfch3=np.reshape(df333, (-1, 1))
dfch4=np.reshape(df444, (-1, 1))
dfch5=np.reshape(df555, (-1, 1))
dfch6=np.reshape(df666, (-1, 1))
dfch7=np.reshape(df777, (-1, 1))


scaler = MinMaxScaler()
df22=scaler.fit_transform(dfch2)
df33=scaler.fit_transform(dfch3)
df44=scaler.fit_transform(dfch4)
df55=scaler.fit_transform(dfch5)
df66=scaler.fit_transform(dfch6)
df77=scaler.fit_transform(dfch7)



df2=pd.DataFrame(df22, columns=['이자보상배율(배)'])
df3=pd.DataFrame(df33, columns=['매출총이익증가율(전분기)(%)'])
df4=pd.DataFrame(df44, columns=['영업이익증가율(전분기)(%)'])
df5=pd.DataFrame(df55, columns=['ROIC(%)'])
df6=pd.DataFrame(df66, columns=['ROA(당기순이익)(%)'])
df7=pd.DataFrame(df77, columns=['EV/EBITDA(직전4분기)(배)'])

df9=pd.DataFrame(df999, columns=['시가(원)'])

result=pd.concat([df2,df3,df4,df5,df6,df7,df9], axis=1)


print(result)

result.to_excel("result.xlsx")

      이자보상배율(배)  매출총이익증가율(전분기)(%)  영업이익증가율(전분기)(%)   ROIC(%)  ROA(당기순이익)(%)  \
0      0.000187          0.001627         0.000275  0.190145       0.182260   
1      0.000188          0.001823         0.000366  0.189788       0.183847   
2      0.000202          0.002238         0.000685  0.190570       0.180980   
3      0.000197          0.001621         0.000244  0.189900       0.177203   
4      0.000228          0.002046         0.000589  0.190547       0.180595   
...         ...               ...              ...       ...            ...   
1551   0.000494          0.001666         0.000359  0.189987       0.180425   
1552   0.000175          0.002364         0.000546  0.190182       0.176263   
1553   0.000174          0.001045         0.000140  0.189435       0.177866   
1554   0.000179          0.003302         0.001093  0.190664       0.180117   
1555   0.000494          0.001214         0.000187  0.189638       0.175091   

      EV/EBITDA(직전4분기)(배)  시가(원)  
0               

In [46]:
df_result = pd.read_excel(os.getcwd()+'\\'+"df_result.xlsx") # 실제 최종 테스트 데이터 (현 시점 가장 최근의 데이터 3월 분기 데이터)
                                                             # 다만 시가의 상승을 맞출려고 하다보니 학습데이터와 다르게 시가는 포함되어 있지 않고 나머지 6개의 데이터만 가지고 시가를예측하도록 해둠
#실제 최종 테스트 데이터 전처리
df222=z_score_normalize(df_result['이자보상배율(배)'])
df333=z_score_normalize(df_result['매출총이익증가율(전분기)(%)'])
df444=z_score_normalize(df_result['영업이익증가율(전분기)(%)'])
df555=z_score_normalize(df_result['ROIC(%)'])
df666=z_score_normalize(df_result['ROA(당기순이익)(%)'])
df777=z_score_normalize(df_result['EV/EBITDA(직전4분기)(배)'])


dfch2=np.reshape(df222, (-1, 1))
dfch3=np.reshape(df333, (-1, 1))
dfch4=np.reshape(df444, (-1, 1))
dfch5=np.reshape(df555, (-1, 1))
dfch6=np.reshape(df666, (-1, 1))
dfch7=np.reshape(df777, (-1, 1))


scaler = MinMaxScaler()
df22=scaler.fit_transform(dfch2)
df33=scaler.fit_transform(dfch3)
df44=scaler.fit_transform(dfch4)
df55=scaler.fit_transform(dfch5)
df66=scaler.fit_transform(dfch6)
df77=scaler.fit_transform(dfch7)



df2=pd.DataFrame(df22, columns=['이자보상배율(배)'])
df3=pd.DataFrame(df33, columns=['매출총이익증가율(전분기)(%)'])
df4=pd.DataFrame(df44, columns=['영업이익증가율(전분기)(%)'])
df5=pd.DataFrame(df55, columns=['ROIC(%)'])
df6=pd.DataFrame(df66, columns=['ROA(당기순이익)(%)'])
df7=pd.DataFrame(df77, columns=['EV/EBITDA(직전4분기)(배)'])


df_result=pd.concat([df2,df3,df4,df5,df6,df7], axis=1)


print(df_result)



     이자보상배율(배)  매출총이익증가율(전분기)(%)  영업이익증가율(전분기)(%)   ROIC(%)  ROA(당기순이익)(%)  \
0     0.103716          0.115665         0.113065  0.416525       0.402686   
1     0.112366          0.056221         0.116428  0.331659       0.356332   
2     0.093024          0.112096         0.127747  0.419793       0.408527   
3     0.088082          0.100207         0.099507  0.398652       0.384339   
4     0.088229          0.121097         0.132844  0.738285       0.400315   
..         ...               ...              ...       ...            ...   
171   0.087051          0.135227         0.173077  0.402219       0.382492   
172   0.125905          0.143469         1.000000  0.430252       0.377474   
173   0.087253          0.106776         0.153944  0.459388       0.391653   
174   0.136230          0.116479         0.132793  0.381495       0.411597   
175   0.086360          0.103208         0.071461  0.374559       0.368513   

     EV/EBITDA(직전4분기)(배)  
0               0.024008  
1        

In [51]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score , recall_score , confusion_matrix

feature_cols = ['이자보상배율(배)','매출총이익증가율(전분기)(%)','영업이익증가율(전분기)(%)','ROIC(%)','ROA(당기순이익)(%)','EV/EBITDA(직전4분기)(배)']
label_cols = ['시가(원)']

X = result[feature_cols]
result = result.astype({'시가(원)': 'int'})
y = result[label_cols]

X_train, X_test, y_train, y_test = train_test_split( #학습데이터 중 30%를 테스트 데이터로 사용하고 데이터는 차례대로 사용하지 않고 섞어서 사용함
    X,
    y,
    test_size=0.3,
    shuffle=True,
)
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)# 특징점이 너무 많아 굴곡이 심해 그 부분을 감소시키기 위해 추가함

model = Sequential()

#model.add(Dense(32, activation="relu", input_shape=(X_train.shape[1],)))
#model.add(Dense(32, activation="relu", input_shape=(X_train.shape[1],)))
# model.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
# model.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
# model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
# model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(256, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(256, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation="sigmoid"))
#model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"]) #학습데이터를 통한 모델 학습, 학습데이터라고 하지만 학습데이터,테스트 데이터로 나뉘어져 모델을 완성시키는 역활임
model.fit(X_train, y_train, epochs=30)
y_pred = model.predict(df_result.astype(float))# 따로 전처리한 최종 테스트 데이터로 결과 예측
model.summary()

pd.DataFrame(y_pred).to_excel("y_pred1.xlsx")
pd.DataFrame(X_test).to_excel("X_test1.xlsx")



Epoch 1/30
35/35 [==============================] - 1s 2ms/step - loss: 0.6893 - acc: 0.5583
Epoch 2/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6883 - acc: 0.5583
Epoch 3/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6873 - acc: 0.5583
Epoch 4/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6869 - acc: 0.5583
Epoch 5/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6873 - acc: 0.5583
Epoch 6/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6866 - acc: 0.5583
Epoch 7/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6862 - acc: 0.5583
Epoch 8/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6874 - acc: 0.5592
Epoch 9/30
35/35 [==============================] - 0s 2ms/step - loss: 0.6873 - acc: 0.5592
Epoch 10/30
35/35 [==============================] - 0s 1ms/step - loss: 0.6871 - acc: 0.5583
Epoch 11/30
35/35 [==============================] - 0s 1ms/step - lo

In [52]:
df_pred=pd.DataFrame(y_pred) #최종 테스트 데이터로 예측한 시가의 증가 확률의 데이터 프레임
df_fin=pd.concat([df_pred,df_name],axis=1)#현재 데이터 프레임의 구분을 위해 원래 회사명을 옆에 붙여줌
df_fin=df_fin.sort_values(by=0 ,ascending=False)#내림차순

rank=df_fin[0:20]#상위 20개만 슬라이싱

print(rank)

pd.DataFrame(df_fin).to_excel("df_fin.xlsx")
pd.DataFrame(rank).to_excel("df_rank.xlsx")
    

            0      name
54   0.777512      강원랜드
65   0.703561      펄어비스
139  0.702328      에스티팜
42   0.678154      엘앤에프
109  0.648075   솔루스첨단소재
96   0.645054      한전기술
3    0.644459  삼성바이오로직스
35   0.626289    에코프로비엠
168  0.624184     파라다이스
36   0.619033  셀트리온헬스케어
72   0.610332   일진머티리얼즈
34   0.602116    포스코케미칼
60   0.595619    카카오게임즈
7    0.594209       카카오
30   0.586467       하이브
24   0.551927      크래프톤
82   0.527513    한미사이언스
41   0.470238       넷마블
44   0.428857    SK바이오팜
4    0.410148     NAVER


In [49]:


X = result[feature_cols]
result = result.astype({'시가(원)': 'int'})
y= result[label_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    shuffle=True,
)

clf = LogisticRegression()
clf.fit(
X_train,
y_train.values.ravel(),
)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

y_pred = model.predict(X_test)




0.5546038543897216
15/15 [==============================] - 0s 926us/step


In [50]:
clf = make_pipeline(StandardScaler(), SVC(gamma="auto"))
clf.fit(
X_train,
y_train.values.ravel(),
)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5588865096359743


In [30]:
knn = KNeighborsClassifier()
knn.fit(
X_train,
y_train.values.ravel(),
)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5417558886509636


In [31]:
clf = RandomForestClassifier()
clf.fit(
X_train,
y_train.values.ravel(),
)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.5246252676659529
